In [113]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [114]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [115]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

## **전처리**

In [116]:
trn_drop_cols = ['PRODUCT_ID', 'TIMESTAMP',                                             
                 'X_934', 'X_935', 'X_936', 'X_937', 'X_2628', 'X_2629', 'X_2630',
                 'X_2631', 'X_2632', 'X_2633', 'X_2634', 'X_2635', 'X_2636', 'X_2637',
                 'X_2638', 'X_2639', 'X_2640', 'X_2641', 'X_2642', 'X_2643', 'X_2644',
                 'X_2645', 'X_2646', 'X_2647', 'X_2648', 'X_2649', 'X_2650', 'X_2651',
                 'X_2652', 'X_2653', 'X_2654', 'X_2655', 'X_2656', 'X_2657', 'X_2658',
                 'X_2659', 'X_2660', 'X_2661', 'X_2662', 'X_2663', 'X_2664', 'X_2665',
                 'X_2666', 'X_2667', 'X_2668', 'X_2669', 'X_2670', 'X_2671', 'X_2672',
                 'X_2673', 'X_2674', 'X_2675', 'X_2676', 'X_2677', 'X_2678', 'X_2679',
                 'X_2680', 'X_2681', 'X_2682', 'X_2683', 'X_2684', 'X_2685', 'X_2686',
                 'X_2687', 'X_2688', 'X_2689', 'X_2690', 'X_2691', 'X_2692', 'X_2693',
                 'X_2694', 'X_2695', 'X_2696', 'X_2697', 'X_2698', 'X_2699', 'X_2838',
                 'X_2844', 'X_2872', 'X_2873', 'X_2874', 'X_2875']                      # 카운트 0인 X_n 컬럼 drop
for col in trn_drop_cols:
    train_df.drop([col], axis=1, inplace=True)

tst_drop_cols = ['PRODUCT_ID', 'TIMESTAMP',
                 'X_934', 'X_935', 'X_936', 'X_937', 'X_2628', 'X_2629', 'X_2630',
                 'X_2631', 'X_2632', 'X_2633', 'X_2634', 'X_2635', 'X_2636', 'X_2637',
                 'X_2638', 'X_2639', 'X_2640', 'X_2641', 'X_2642', 'X_2643', 'X_2644',
                 'X_2645', 'X_2646', 'X_2647', 'X_2648', 'X_2649', 'X_2650', 'X_2651',
                 'X_2652', 'X_2653', 'X_2654', 'X_2655', 'X_2656', 'X_2657', 'X_2658',
                 'X_2659', 'X_2660', 'X_2661', 'X_2662', 'X_2663', 'X_2664', 'X_2665',
                 'X_2666', 'X_2667', 'X_2668', 'X_2669', 'X_2670', 'X_2671', 'X_2672',
                 'X_2673', 'X_2674', 'X_2675', 'X_2676', 'X_2677', 'X_2678', 'X_2679',
                 'X_2680', 'X_2681', 'X_2682', 'X_2683', 'X_2684', 'X_2685', 'X_2686',
                 'X_2687', 'X_2688', 'X_2689', 'X_2690', 'X_2691', 'X_2692', 'X_2693',
                 'X_2694', 'X_2695', 'X_2696', 'X_2697', 'X_2698', 'X_2699', 'X_2838',
                 'X_2844', 'X_2872', 'X_2873', 'X_2874', 'X_2875']                      # test도 train과 동일하게 drop
for col in tst_drop_cols:
    test_df.drop([col], axis=1, inplace=True)

In [117]:
drop_cols = ['LINE', 'PRODUCT_CODE']

for i in drop_cols:
    le = LabelEncoder()
    le = le.fit(train_df[i])
    train_df[i] = le.transform(train_df[i])
    
    for label in np.unique(test_df[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_df[i] = le.transform(test_df[i]) 
print('Done.')

Done.


In [118]:
train_df.head(1)

,Y_Class,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,1,0.533433,2,0,NaN,NaN,NaN,NaN,NaN,NaN,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN


## **모델링**

In [119]:
train_x = train_df.drop(columns=['Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df

In [107]:
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=2, random_state=37)
xgb.fit(train_x, train_y)
pred = xgb.predict(test_x)

submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = pred

submit.to_csv('./submit_3.csv', index=False)

c:\Users\LG\anaconda3\envs\py39_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\LG\anaconda3\envs\py39_env\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[03:08:32] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## **결과**

In [111]:
submit_3 = pd.read_csv('C:\\VSC\\LG_Aimers\\submit_3.csv')
submit_3.Y_Class.value_counts()

1    264
0     34
2     12
Name: Y_Class, dtype: int64